![banner](./assets/banner.jpeg)

> You can open this notebook in Colab by clicking the Colab icon. Colab provides GPU for free. You can also run this notebook locally by installing the dependencies listed in `requirements.txt`.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/osbm/unet_explainer/blob/main/tutorial-part1.ipynb)

### Preamble with Additional Comments and Explanations

#### Introduction
In the first section of this hands-on workshop, we will explore a working example that employs a pretrained deep learning model for semantic segmentation of prostate images. Specifically, we will focus on the segmentation of 2D multi-slice prostate MR images. For this exercise, we will use the [prostate-158 test dataset](https://zenodo.org/record/6592345), which comprises 19 MRI images of the prostate.

#### Objectives of this Part
1. **Understand the Basics of Semantic Segmentation**: Semantic segmentation involves classifying each pixel in an image as belonging to one of the predefined classes. In the medical imaging context, this is crucial for isolating regions of interest, such as tumors or specific organs like the prostate, from the surrounding tissue.
  
2. **Grasp the Fundamentals of U-Net Architecture**: The U-Net architecture is a convolutional neural network that was initially designed for biomedical image segmentation. It has since been adapted for a variety of segmentation tasks. Understanding its architecture and functionalities will empower you to harness its capabilities for medical imaging applications effectively.

3. **Learn to Predict on Prostate Images Using a Pretrained Model**: Utilizing pretrained models can significantly accelerate the process of medical image analysis. In this part, we will walk you through the steps of using a pretrained U-Net model to perform segmentation on prostate MR images. In the subsequent section, you will learn how to train a U-Net model from scratch for custom applications.

#### Note to Participants
This workshop is designed to be an interactive, hands-on experience. Therefore, internet access will be required to access the Google Colab platform for running the code.

By the end of this section, you should have a foundational understanding of semantic segmentation, the U-Net architecture, and the practical application of these technologies in prostate MR image analysis.

Feel free to ask questions or seek clarification at any point during the workshop. Your active participation is encouraged for a more enriching learning experience.

### Our works

- Karagoz, Ahmet, et al. "Anatomically guided self-adapting deep neural network for clinically significant prostate cancer detection on bi-parametric MRI: a multi-center study." Insights into Imaging 14.1 (2023): 1-11. https://doi.org/10.1186/s13244-023-01439-0

- Karagoz, Ahmet, et al. "Prostate Lesion Estimation using Prostate Masks from Biparametric MRI." arXiv preprint arXiv:2301.09673 (2023). https://doi.org/10.48550/arXiv.2301.09673

#### Installing Required Software and Libraries:

For your convenience, I have created a Python package that encapsulates all the necessary dependencies and libraries required for this tutorial. You can install this package using the command provided below. By doing so, you will automatically install all the libraries that are essential for this workshop.

For those interested in understanding the underlying implementation of the functions and classes used in this tutorial, the source code is publicly available in the [unet_explainer GitHub repository](https://github.com/osbm/unet_explainer/).

#### Note to Participants:

1. **Why a Custom Python Package?**: Packaging the required libraries and dependencies into a single installable unit simplifies the setup process, allowing you to focus more on the tutorial's content rather than troubleshooting installation issues.

2. **Transparency and Extensibility**: The availability of the source code in a public repository offers transparency and provides an opportunity for future customization and improvement. You are encouraged to explore the repository to gain deeper insights into the functionalities provided.

By adhering to the above installation steps, you ensure a smooth and efficient setup, allowing us to dive straight into the core topics of this workshop.

###  Section with Additional Comments and Explanations

#### Installation Command for the Custom Python Package:

If you're curious about the implementation details or wish to inspect the code, please note that this is a public repository. To install the custom Python package and all its dependencies, run the following command in your Google Colab environment:

```bash
!pip install git+https://github.com/osbm/unet_explainer.git
```

#### Note to Participants:

1. **Code Inspection**: The repository is open-source, allowing for code inspection. This offers an additional layer of transparency and enables you to understand the inner workings of the package, which could be beneficial for your own projects.

2. **Execution Environment**: This command is specifically designed to be run in a Google Colab environment. If you're using a different Python environment, additional setup steps may be required.

3. **Internet Access Requirement**: This command fetches data from an online GitHub repository. Ensure you have an active internet connection before executing the command.

By successfully executing this command, you will have installed all the necessary dependencies, setting the stage for the remainder of this hands-on workshop.## Install requirements:

I have made a python package that you can install with the below command. This package will also automatically install all the libraries needed for this tutorial. If you wanto see how the functions and the classes are implemented, you can check the source code in [unet_explainer](https://github.com/osbm/unet_explainer/) repository.

In [ ]:
# This is a public repository, please inspect the code if you are curious about the implementation details.
!pip install git+https://github.com/osbm/unet_explainer.git

# Imports

Import all the used built-in, custom and third-party libraries to use in this notebook.

In [ ]:
# lets import our functions and classes
from unet_pytorch import (
    ProstateDataset,     # Our custom dataset class
    print_model_info,    # Function to print pytorch model info
    predict,             # Function to predict on a single image
    set_seed,            # Function to set seed for reproducibility
    plot_predictions,    # Function to plot predictions
    plot_one_example,    # Function to plot one example
)

# third party libraries
import torch                                  # pytorch deep learning framework
from torch.utils.data import DataLoader       # dataloader class from pytorch to load data

import monai                                  # monai medical imaging framework (built on top of pytorch)
from monai.networks.nets import UNet          # unet model from monai (there are other models that you use with a single line)

import albumentations as A                    # albumentations library for image augmentation and preprocessing
from albumentations.pytorch import ToTensorV2 # albumentations class to convert images to tensors

# built-in libraries
import os  # os library to work with files and directories

#### Downloading the Dataset and Pretrained Model:

For ease of access, both the preprocessed version of the dataset and the pretrained model have been hosted on a public Hugging Face dataset repository. The necessary commands for downloading these resources are provided below.

Additionally, it's important to note that the dataset has undergone specific preprocessing, which you can review in detail through [this script](https://github.com/osbm/unet_explainer/blob/main/scripts/preprocess_data.py).

#### Preprocessing Summary:

The original Prostate158 dataset comprises 3D volume images and masks of patients' prostates. However, since our model focuses on 2D segmentation, certain preprocessing steps were necessary. Specifically, slices that contain segmentation masks covering less than 6% of the image were filtered out. This threshold is arbitrary, and you are encouraged to experiment with different values to create new datasets.

#### Rationale for Preprocessing:

The objective of this filtering is to exclude slices that do not contain a significant portion of the prostate, as they would not be useful for training the model.

#### Note to Participants:

1. **Preprocessed Dataset**: The dataset you will be working with has already undergone this preprocessing. It's designed to be immediately usable for this workshop.
  
2. **Customization**: If you wish to experiment with different preprocessing conditions, the provided script will be a useful reference.
  
3. **Training Considerations**: The exclusion of slices with minimal prostate segmentation is an arbitrary but crucial part of the data preparation. It's designed to improve the model's focus on relevant image features during training.

Proceeding with the dataset download will equip you with the necessary data for the hands-on segments of this workshop.

#### Downloading and Unzipping the Dataset:

The following code block ensures that the required dataset is downloaded and unzipped for use in this notebook. If the dataset directory (`data`) does not already exist, the code will download it from the specified Hugging Face repository and unzip the data files.

```python
if not os.path.exists('data'):  # Check if the data directory exists
    # Download the dataset from the Hugging Face repository
    !wget -q https://huggingface.co/datasets/osbm/unet-explainer-data/resolve/main/data.zip
    # Unzip the downloaded dataset
    !unzip -q data.zip
```

#### Note to Participants:

1. **Conditional Check**: The `if not os.path.exists('data')` condition ensures that the dataset is downloaded only if it is not already present, saving bandwidth and time.

2. **Quiet Mode**: The `-q` flag in both `wget` and `unzip` commands indicates 'quiet mode,' which suppresses output messages. This is to keep the notebook uncluttered.

3. **Data Source**: The dataset is hosted on a Hugging Face repository, a platform commonly used for sharing machine learning datasets and models.

4. **Local Storage**: The data will be stored in a directory named `data`, which will be created in the same location as this notebook.

By executing this block, you will either confirm the presence of the necessary data or download it, thereby ensuring that all subsequent sections can proceed without data-related hindrances.

In [ ]:
if not os.path.exists('data'): # download data if it doesn't exist
    # download data
    !wget -q https://huggingface.co/datasets/osbm/unet-explainer-data/resolve/main/data.zip
    # unzip data
    !unzip -q data.zip

# -q flag means quiet, so you won't see any output

In [ ]:
# also download our pretrained model
!wget -q https://huggingface.co/datasets/osbm/unet-explainer-data/resolve/main/best_model.pth

# Reproducibility

We should set an RNG seed for reproducibility. This way we can get the same results on each run. This is important for debugging and comparing different models. Also is useful if you want to prove that you didnt forge your results.

> Warning: Total deterministic behavior is not guaranteed between PyTorch releases, individual commits or different platforms. Furthermore, results may not be reproducible between CPU and GPU executions, even when using identical seeds. For this reason it is recommended to also share python version, exact PyTorch version and platform (OS, GPU etc.) when reporting results.

In [ ]:
set_seed(42)

#### Applying Transformations to Images and Masks:

Data transformations are essential steps in the preprocessing pipeline, especially for machine learning tasks involving images. For this workshop, we will focus on a set of specific transformations: resizing the images and masks to a fixed dimension, converting them into tensors, and normalizing them. We will employ the Albumentations library to achieve these objectives. Albumentations is a fast and flexible library designed for augmenting images for machine learning tasks.

#### Commonly Used Transformation Libraries:

1. **Torchvision**: Native to PyTorch, this library offers a wide range of transformations and is commonly used in various computer vision tasks.
  
2. **Albumentations**: Known for its speed and flexibility, Albumentations provides a comprehensive range of image transformations, making it suitable for a variety of applications.
  
3. **MONAI**: Specially designed for medical imaging, MONAI offers a set of transformations that are particularly useful for tasks like segmentation in medical images.

#### Note to Participants:

1. **Why Transformations?**: Applying transformations like resizing and normalization makes the data compatible with the model architecture and can also enhance model performance.

2. **Choice of Library**: The use of Albumentations in this tutorial is not limiting; you can easily adapt the code to use Torchvision or MONAI based on your specific needs.

3. **Medical Imaging Specificity**: If you are dealing exclusively with medical images, MONAI might offer some specialized transformations that could be beneficial.

4. **Tensors**: Converting images to tensors is a requisite step for using them in PyTorch-based models. This ensures that the data is in a format compatible with the computational libraries.

By understanding and applying these transformations, you are setting the stage for the effective training and evaluation of your machine learning model on the prostate MR images.

In [ ]:
image_size = 256
valid_transforms = A.Compose([
    A.Resize(height=image_size, width=image_size),
    ToTensorV2(),
])
# all we are doing here is resizing the image to 256x256 and converting it to a tensor

#### Dataset and DataLoader in PyTorch:

When working with PyTorch, it's common practice to define a custom dataset class and then employ a DataLoader to efficiently handle this dataset. This pattern is central to how PyTorch manages data for machine learning tasks.

#### Components of a PyTorch Dataset:

1. **Number of Samples**: The dataset class should provide information about the total number of samples in the dataset.
  
2. **Single Sample Retrieval**: The class should define a method for fetching a single sample from the dataset, often via indexing.

3. **Data Transformations**: The dataset class is also responsible for applying any necessary transformations to the samples. This is the stage where most of your data processing logic will be applied.

#### DataLoader:

The DataLoader takes this dataset class as an input and provides an iterable over the dataset. It handles batching, shuffling, and loading data in parallel, among other functionalities.

#### Note to Participants:

1. **Why a Custom Dataset Class?**: Custom dataset classes provide the flexibility to define specific ways to load and preprocess data, which is particularly useful for specialized tasks like medical imaging.

2. **DataLoader Utility**: DataLoader is essential for handling large datasets and for facilitating more complex operations like mini-batch gradient descent.

3. **Transformation Integration**: The dataset class is often where data transformations are implemented, ensuring that each sample is correctly preprocessed before being fed into the model.

4. **Efficiency**: This architecture is designed to be highly efficient, enabling faster model training and evaluation.

By understanding the role and structure of the Dataset and DataLoader in PyTorch, you are better equipped to manage data effectively for machine learning tasks, including the segmentation of prostate MR images in this workshop.

In [ ]:
test_ds = ProstateDataset(folder='data/test', transform=valid_transforms)

In [ ]:
len(test_ds) # number of samples in the dataset

In [ ]:
example_image, example_mask = test_ds[24]
example_image.shape, example_mask.shape # shapes will be [channels, height, width] for both image and mask

#### Visualizing an Example Sample:

Before diving into model training and evaluation, it's beneficial to inspect an example sample from our dataset. This helps in understanding the kind of data you will be working with and ensures that it has been loaded and preprocessed correctly.

#### Interactive Exercise:

In this interactive segment, you will be given the opportunity to visualize an example image and its corresponding mask from the dataset. The goal is to familiarize yourself with the data and understand its characteristics.


```python
# Code to plot an example image and its mask
plot_one_example(example_image, example_mask)

```
#### Note to Participants:

1. **Understanding Data**: Visual inspection can provide valuable insights into the quality and characteristics of the data, such as the level of detail in the images and masks.

2. **Preprocessing Check**: This is also an opportunity to verify that any preprocessing steps (like resizing or normalization) have been correctly applied to the dataset.

3. **Interactive Learning**: This exercise aims to make the learning experience more engaging and hands-on, allowing you to actively participate in the workshop.

In the cell below, you'll find the code to plot an example image and its corresponding mask. Execute the cell to visualize the sample.

In [ ]:
# lets see our example sample
plot_one_example(example_image, example_mask)

In [ ]:
test_loader = DataLoader(test_ds, batch_size=16, shuffle=True)

#### Understanding PyTorch DataLoader and Batching:

In PyTorch, a DataLoader serves as the mechanism to fetch batches of samples from the dataset for training or evaluation. Several options can be customized within the DataLoader, such as:

- **Batch Size**: The number of samples per batch.
- **Number of Workers**: The number of subprocesses to use for data loading.
- **Shuffling**: Whether to shuffle the data before loading it.
- **Additional Parameters**: Other options like prefetching data, applying custom collate functions, etc., can also be specified.

#### What is a Batch?

A batch is essentially a subset of the dataset, grouped together for the purpose of model training. Rather than updating the model weights based on the gradient of a single sample, batches allow for the gradients to be computed across multiple samples. The model's weights are then updated using the average gradient of the entire batch.

#### Advantages of Batching:

1. **Efficiency**: Computing gradients and updating model weights for multiple samples in parallel is computationally more efficient.
  
2. **Generalization**: Averaging the gradients across multiple samples can smooth out the noise, aiding in model generalization.

3. **Example**: If your dataset contains 1000 samples and you set the batch size to 10, the DataLoader will create 100 batches, each containing 10 samples.

#### Note to Participants:

1. **Choice of Batch Size**: The batch size is a hyperparameter that can impact both model performance and training speed. It's often chosen based on available computational resources and empirical performance evaluation.

2. **Number of Workers**: Increasing the number of workers can speed up data loading but may also increase the memory footprint.

3. **Shuffling**: Shuffling the data helps in breaking any inherent order in the dataset, thus aiding in better model generalization.

Understanding the DataLoader's functionality and the concept of batching equips you with the knowledge to efficiently train machine learning models, including the segmentation of prostate MR images in this workshop.

# The Model

Finally, the juicy part. We will use a pretrained U-Net model to predict on our dataset. U-Net is a popular architecture for semantic segmentation. It is a convolutional neural network that is used for image segmentation. It was first introduced by Olaf Ronneberger, Philipp Fischer, and Thomas Brox in the paper U-Net: Convolutional Networks for Biomedical Image Segmentation in 2015.

A U-Net consists of two parts:
- Contracting path (left side of the U)
- Expanding path (right side of the U)

![unet-architecture](./assets/unet-architecture.png)

This picture looks a bit complicated. But it is not. Let's break it down.

The contracting path is a typical convolutional network that consists of repeated application of convolutions, each followed by a rectified linear unit (ReLU) and a max pooling operation. During the contraction, the spatial information is reduced while feature information is increased. The expanding path combines the feature and spatial information through a sequence of up-convolutions and concatenations with high-resolution features from the contracting path.

While getting lower in the network, number of channels are increased and the height and width of the image is decreased.

But we are not completely losing the spatial information. We are storing the spatial information in the expanding path. We are concatenating the feature maps from the contracting path to the feature maps in the expanding path. This way we are combining the feature maps from the contracting path with the spatial information from the expanding path.

Also one hidden benefit of using skip connections is that it helps with vanishing gradient problem. This problem is mostly encountered in deep neural networks. 

<!-- ![vanishing-gradient-problem](./assets/vanishing-gradients.jpeg) -->

<img src="./assets/vanishing-gradients.jpeg" alt="vanishing-gradient-problem" width="500"/>

The gradient signal goes through many layers during backpropagation, and the gradient signal becomes smaller and smaller as it goes deeper and reaches the first layers (closer to the input layer). This is called the vanishing gradient problem. This problem makes training deep networks hard.

### Convolution Operation

The first figure illustrates the convolution operation with example pixel intensities included in each cell. The blue grid represents the input matrix, and the red grid represents the kernel matrix.

- **Input Matrix**: The blue grid contains example pixel intensities, ranging from 0 to 2. These would typically be values derived from an image.
  
- **Kernel Matrix**: The red grid also contains example values. These values are usually optimized during the training of the neural network.

### Max Pooling Operation

The second figure illustrates the max pooling operation, also with example pixel intensities. The blue grid represents the input matrix.

- **Input Matrix**: The blue grid contains example pixel intensities, ranging from 0 to 4. These would typically be values derived from a feature map generated by a previous layer in a neural network.
  
- **Pooling Region**: The green grid represents a 2x2 pooling region. The maximum value within this region would be taken as the output for that region.

Including example numbers in these figures aims to provide a more concrete understanding of how these operations work, especially in the context of pixel intensities in image data. This understanding is crucial for advanced image analysis tasks, such as the prostate MR image segmentation you're focused on in this workshop.

### Revised Section with Additional Comments and Explanations

#### Defining the U-Net Model:

In this section, we instantiate the U-Net model using the MONAI library. While MONAI is specialized for medical imaging tasks, it returns a PyTorch model, allowing seamless integration with existing PyTorch workflows.

Here's a breakdown of the model parameters:

```python
model = UNet(
    spatial_dims=2,                  # Specifies that the image is 2D
    in_channels=1,                   # Indicates the number of input channels (T2-weighted MRI images in this case)
    out_channels=3,                  # Number of output channels corresponding to 3 labels
    channels=[16, 32, 64, 128, 256, 512],  # Number of channels at each layer during contraction
    strides=(2, 2, 2, 2, 2),         # Strides for the convolutional layers
    num_res_units=4,                 # Number of residual units
    dropout=0.15,                    # Dropout rate to prevent overfitting
)
```

#### Note to Participants:

1. **Spatial Dimensions**: The `spatial_dims=2` parameter is indicative of the 2D nature of the images we're working with.

2. **Input and Output Channels**: `in_channels=1` and `out_channels=3` specify the number of input and output channels, which correspond to the T2-weighted MRI images and the 3 segmentation labels, respectively.

3. **Channels and Strides**: The `channels` and `strides` parameters define the architecture of the convolutional layers in the network.

4. **Residual Units**: `num_res_units=4` indicates the use of residual connections, which can help the network learn more complex functions and alleviate the vanishing gradient problem.

5. **Dropout**: The `dropout=0.15` parameter helps in preventing overfitting by randomly setting a fraction of the input units to 0 during training.

6. **PyTorch Compatibility**: Since MONAI returns a PyTorch model, you can employ standard PyTorch functionalities like loss functions, optimizers, etc., for training and evaluation.

Understanding these parameters will help you grasp the architecture and functionality of the U-Net model, which is crucial for tasks like the segmentation of prostate MR images in this workshop.

In [ ]:
model = UNet(
    spatial_dims=2, # 2d image
    in_channels=1,  # we only used  T2 weighed MRI images
    out_channels=3, # 3 labels
    channels=[16, 32, 64, 128, 256, 512], # number of channels to use while contracting
    strides=(2, 2, 2, 2, 2), # CNN strides
    num_res_units=4, # residual connections
    dropout=0.15, # dropout rate
)
# the monai library returns a pytorch model, so we can use it as a pytorch model

In [ ]:
print_model_info(model)

# Selecting a device to run the model

A device is used for accelerating the training process. You need to be aware of your devices memory capacity. This notebook runs smoothly in **Colab T4 GPU**.

Pytorch supports both CPU and GPU. You can select which device to use with the below code. If you have a GPU, you should use it. Because it is much faster than CPU. But if you don't have a GPU, you can still run this notebook on CPU. We are just making inference after all.

> You can also use a Apple MPS (Metal Performance Shaders) to train models on Apple devices (especially if you have M1 or M2 chips instead of GPU cards). Simply change below line with:

```python
device = torch.device('mps' if torch.mps.is_available() else 'cpu')
```

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# now our model class only has the logic and operations, we need to load the weights
model.load_state_dict(torch.load('best_model.pth', map_location=device))
# map_location argument is used to place model weights on the current device (cpu or gpu) directly
# normally if you would want to move a model, you would call:
# model.to(device)

#### Making Predictions with the Model:

In this section, we will utilize the `predict` function to generate predictions on the test set. This function not only produces the predicted outputs but can also calculate various evaluation scores if needed.

Here's the code snippet to perform the prediction:

```python
x, y, y_hat = predict(
    model,                      # The trained U-Net model
    test_loader=test_loader,    # DataLoader object for the test set
    device=device,              # Computing device (CPU or GPU)
    final_activation="softmax", # Final activation function
    calculate_scores=True,      # Whether to calculate evaluation scores
)
```

#### Note to Participants:

1. **Model**: The `model` parameter refers to the trained U-Net model that will be used for making predictions.

2. **Test Loader**: `test_loader` is the DataLoader object containing the test dataset.

3. **Device**: The `device` parameter specifies the hardware where the computation will be performed, be it CPU or GPU.

4. **Final Activation**: The `final_activation="softmax"` parameter indicates that a Softmax activation function will be applied to the model's output to produce probabilities.

5. **Calculate Scores**: The `calculate_scores=True` parameter indicates that evaluation metrics will also be computed, in addition to generating predictions.

By understanding this prediction step, you're well-prepared to evaluate how well the U-Net model performs on unseen data, a crucial aspect in the realm of medical image segmentation and your ongoing workshop on prostate MR images.

In [ ]:
# lets make the model prediction on the test set using the predict function
x, y, y_hat = predict(
    model,
    test_loader=test_loader,
    device=device,
    final_activation="softmax",
    calculate_scores=True,
)

In [ ]:
# lets take a look at our output tensor shapes
print(x.shape, y.shape, y_hat.shape)

In [ ]:
plot_predictions(x, y, y_hat, num_examples_to_plot=5)